In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241211_01_SparkSQL_SQLtest").getOrCreate()
spark

24/12/11 10:26:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
from pyspark.sql import Row

user_data = [
    Row(user_id=1, username='A', address='서울'),
    Row(user_id=2, username='B', address='대전'),
    Row(user_id=3, username='C', address='경기도'),
    Row(user_id=4, username='D', address=None),
    Row(user_id=5, username='E', address=None),
    Row(user_id=6, username='F', address='서울'),
    Row(user_id=7, username='G', address='경기도'),
    Row(user_id=8, username='H', address='대구'),
    Row(user_id=9, username='I', address='부산'),
    Row(user_id=10, username='J', address='전주'),
    Row(user_id=11, username='K', address='광주')
]

In [6]:
user_df = spark.createDataFrame(user_data)
user_df.createOrReplaceTempView('users')

In [7]:
books_data = [
    Row(book_id=1, title="Book A", author_fname="John", author_lname="Doe", pages=300, released_year=2005, stock_quantity=55),
    Row(book_id=2, title="Book B", author_fname="Jane", author_lname="Smith", pages=250, released_year=2010, stock_quantity=40),
    Row(book_id=3, title="Book C", author_fname="Emily", author_lname="Jones", pages=180, released_year=2015, stock_quantity=20),
    Row(book_id=4, title="Book D", author_fname="Chris", author_lname="Brown", pages=320, released_year=2012, stock_quantity=75),
    Row(book_id=5, title="Book E", author_fname="Anna", author_lname="Davis", pages=270, released_year=2008, stock_quantity=35)
]


In [8]:
books_df = spark.createDataFrame(books_data)
books_df.createOrReplaceTempView('books')

In [11]:
query_users = '''
SELECT * 
FROM users;
'''
spark.sql(query_users).show()

+-------+--------+-------+
|user_id|username|address|
+-------+--------+-------+
|      1|       A|   서울|
|      2|       B|   대전|
|      3|       C| 경기도|
|      4|       D|   null|
|      5|       E|   null|
|      6|       F|   서울|
|      7|       G| 경기도|
|      8|       H|   대구|
|      9|       I|   부산|
|     10|       J|   전주|
|     11|       K|   광주|
+-------+--------+-------+



In [12]:
query_users2 = '''
SELECT username,
    IF (address IS NULL, '주소없음', address) as address
FROM users;
'''
spark.sql(query_users2).show()

+--------+--------+
|username| address|
+--------+--------+
|       A|    서울|
|       B|    대전|
|       C|  경기도|
|       D|주소없음|
|       E|주소없음|
|       F|    서울|
|       G|  경기도|
|       H|    대구|
|       I|    부산|
|       J|    전주|
|       K|    광주|
+--------+--------+



In [16]:
query_users3 = '''
SELECT address, 
    IF(address IN ('경기도', '서울'), '수도권', '지방') AS region 
FROM users;
'''
spark.sql(query_users3).show()

+-------+------+
|address|region|
+-------+------+
|   서울|수도권|
|   대전|  지방|
| 경기도|수도권|
|   null|  지방|
|   null|  지방|
|   서울|수도권|
| 경기도|수도권|
|   대구|  지방|
|   부산|  지방|
|   전주|  지방|
|   광주|  지방|
+-------+------+



In [21]:
# books table
# stock_quantity > 50 '재고 많음', >= 30 '재고 중간', '재고 없음'

books_sql = '''
SELECT stock_quantity, 
       IF(stock_quantity >= 50, '재고 많음', 
          IF(stock_quantity >= 30, '재고 중간', '재고 없음')) AS quantity_level
FROM books;
'''
spark.sql(books_sql).show()

+--------------+--------------+
|stock_quantity|quantity_level|
+--------------+--------------+
|            55|     재고 많음|
|            40|     재고 중간|
|            20|     재고 없음|
|            75|     재고 많음|
|            35|     재고 중간|
+--------------+--------------+



In [22]:
books_sql_1= '''
SELECT stock_quantity, 
   CASE 
       WHEN stock_quantity >= 50 THEN '재고 많음'
       WHEN stock_quantity >= 30 THEN '재고 중간'
       ELSE '재고 부족'
   END AS quantity_level
FROM books;
'''
spark.sql(books_sql_1).show()

+--------------+--------------+
|stock_quantity|quantity_level|
+--------------+--------------+
|            55|     재고 많음|
|            40|     재고 중간|
|            20|     재고 부족|
|            75|     재고 많음|
|            35|     재고 중간|
+--------------+--------------+



# 실행 계획 비교

In [24]:
spark.sql(books_sql).explain()

== Physical Plan ==
*(1) Project [stock_quantity#21L, if ((stock_quantity#21L >= 50)) 재고 많음 else if ((stock_quantity#21L >= 30)) 재고 중간 else 재고 없음 AS quantity_level#147]
+- *(1) Scan ExistingRDD[book_id#15L,title#16,author_fname#17,author_lname#18,pages#19L,released_year#20L,stock_quantity#21L]




In [25]:
spark.sql(books_sql_1).explain()

== Physical Plan ==
*(1) Project [stock_quantity#21L, CASE WHEN (stock_quantity#21L >= 50) THEN 재고 많음 WHEN (stock_quantity#21L >= 30) THEN 재고 중간 ELSE 재고 부족 END AS quantity_level#150]
+- *(1) Scan ExistingRDD[book_id#15L,title#16,author_fname#17,author_lname#18,pages#19L,released_year#20L,stock_quantity#21L]




In [34]:
books_sql_2 = '''
select distinct author_lname from books;
'''
spark.sql(books_sql_2).explain()
spark.sql(books_sql_2).show()

== Physical Plan ==
*(2) HashAggregate(keys=[author_lname#18], functions=[])
+- Exchange hashpartitioning(author_lname#18, 200), ENSURE_REQUIREMENTS, [id=#270]
   +- *(1) HashAggregate(keys=[author_lname#18], functions=[])
      +- *(1) Project [author_lname#18]
         +- *(1) Scan ExistingRDD[book_id#15L,title#16,author_fname#17,author_lname#18,pages#19L,released_year#20L,stock_quantity#21L]


+------------+
|author_lname|
+------------+
|       Jones|
|       Davis|
|       Smith|
|         Doe|
|       Brown|
+------------+



In [33]:
books_sql_3= '''
SELECT author_lname, count(*)
FROM books
GROUP BY author_lname;
'''
spark.sql(books_sql_3).explain()
spark.sql(books_sql_3).show()

== Physical Plan ==
*(2) HashAggregate(keys=[author_lname#18], functions=[count(1)])
+- Exchange hashpartitioning(author_lname#18, 200), ENSURE_REQUIREMENTS, [id=#223]
   +- *(1) HashAggregate(keys=[author_lname#18], functions=[partial_count(1)])
      +- *(1) Project [author_lname#18]
         +- *(1) Scan ExistingRDD[book_id#15L,title#16,author_fname#17,author_lname#18,pages#19L,released_year#20L,stock_quantity#21L]


+------------+--------+
|author_lname|count(1)|
+------------+--------+
|       Jones|       1|
|       Davis|       1|
|       Smith|       1|
|         Doe|       1|
|       Brown|       1|
+------------+--------+



# 데이터 변경

In [35]:
# books 테이블 데이터에 borrowed_by 추가
books_data_with_user = [
    Row(book_id=1, title="Book A", author_fname="John", author_lname="Doe", pages=300, released_year=2005, stock_quantity=55, borrowed_by=1),
    Row(book_id=2, title="Book B", author_fname="Jane", author_lname="Smith", pages=250, released_year=2010, stock_quantity=40, borrowed_by=2),
    Row(book_id=3, title="Book C", author_fname="Emily", author_lname="Jones", pages=180, released_year=2015, stock_quantity=20, borrowed_by=3),
    Row(book_id=4, title="Book D", author_fname="Chris", author_lname="Brown", pages=320, released_year=2012, stock_quantity=75, borrowed_by=None),
    Row(book_id=5, title="Book E", author_fname="Anna", author_lname="Davis", pages=270, released_year=2008, stock_quantity=35, borrowed_by=6)
]

# DataFrame 생성
books_df_with_user = spark.createDataFrame(books_data_with_user)

# Temp View 등록
books_df_with_user.createOrReplaceTempView("books")

In [ ]:
# # borrowed_by 컬럼 추가 및 데이터 입력
# updated_books_df = books_df.withColumn(
#     "borrowed_by",
#     when(books_df.book_id == 1, 1)
#     .when(books_df.book_id == 2, 2)
#     .when(books_df.book_id == 3, 3)
#     .when(books_df.book_id == 4, lit(None))
#     .when(books_df.book_id == 5, 6)
#     .otherwise(None)
# )

In [36]:
books_sql= '''
SELECT *
FROM books
'''
spark.sql(books_sql).explain()
spark.sql(books_sql).show()

== Physical Plan ==
*(1) Scan ExistingRDD[book_id#203L,title#204,author_fname#205,author_lname#206,pages#207L,released_year#208L,stock_quantity#209L,borrowed_by#210L]


+-------+------+------------+------------+-----+-------------+--------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+--------------+-----------+
|      1|Book A|        John|         Doe|  300|         2005|            55|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|            40|          2|
|      3|Book C|       Emily|       Jones|  180|         2015|            20|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|            75|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|            35|          6|
+-------+------+------------+------------+-----+-------------+--------------+-----------+



In [38]:
from pyspark.sql.functions import *

In [39]:
# book_id = 3, stock_quantity = 50 으로 바꿈 > 전처리 과정

updated_books_df = books_df_with_user.withColumn(
    "stock_quantity",
    when (books_df_with_user.book_id == 3, 50).otherwise(books_df_with_user.stock_quantity) 
)

updated_books_df.show()

+-------+------+------------+------------+-----+-------------+--------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+--------------+-----------+
|      1|Book A|        John|         Doe|  300|         2005|            55|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|            40|          2|
|      3|Book C|       Emily|       Jones|  180|         2015|            50|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|            75|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|            35|          6|
+-------+------+------------+------------+-----+-------------+--------------+-----------+



In [46]:
# stock_quantity * 10% 증가

updated_books_df = books_df_with_user.withColumn(
    "stock_quantity",
    col("stock_quantity") * 1.10)

#뷰로 등록
updated_books_df.createOrReplaceTempView("books")
spark.sql("select * from books").show()

+-------+------+------------+------------+-----+-------------+-----------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|   stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+-----------------+-----------+
|      1|Book A|        John|         Doe|  300|         2005|60.50000000000001|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|             44.0|          2|
|      3|Book C|       Emily|       Jones|  180|         2015|             22.0|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|             82.5|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|             38.5|          6|
+-------+------+------------+------------+-----+-------------+-----------------+-----------+



# 데이터 저장

In [49]:
# write 의 저장 mode : overwrite, append, ignore, error

updated_books_df.write.csv("data/output/sqltest_updated_books.csv", header = True, mode = "overwrite")
user_df.write.csv("data/output/sqltest_user_df.csv", header = True, mode = "overwrite")

In [53]:
# 데이터 불러오기
updated_books_df1 = spark.read.csv("data/output/sqltest_updated_books.csv", header = True)
updated_books_df1.show()

+-------+------+------------+------------+-----+-------------+-----------------+-----------+
|book_id| title|author_fname|author_lname|pages|released_year|   stock_quantity|borrowed_by|
+-------+------+------------+------------+-----+-------------+-----------------+-----------+
|      3|Book C|       Emily|       Jones|  180|         2015|             22.0|          3|
|      4|Book D|       Chris|       Brown|  320|         2012|             82.5|       null|
|      5|Book E|        Anna|       Davis|  270|         2008|             38.5|          6|
|      1|Book A|        John|         Doe|  300|         2005|60.50000000000001|          1|
|      2|Book B|        Jane|       Smith|  250|         2010|             44.0|          2|
+-------+------+------------+------------+-----+-------------+-----------------+-----------+



In [54]:
user_df1 = spark.read.csv("data/output/sqltest_user_df.csv", header = True)
user_df1.show()

+-------+--------+-------+
|user_id|username|address|
+-------+--------+-------+
|      6|       F|   서울|
|      7|       G| 경기도|
|      8|       H|   대구|
|      9|       I|   부산|
|     10|       J|   전주|
|     11|       K|   광주|
|      1|       A|   서울|
|      2|       B|   대전|
|      3|       C| 경기도|
|      4|       D|   null|
|      5|       E|   null|
+-------+--------+-------+



# 조인 실습

In [55]:
# INNER JOIN
join_query = '''
SELECT book_id, title, author_fname, author_lname, username, address
FROM books b INNER JOIN users u ON b.borrowed_by = u.user_id;
'''
spark.sql(join_query).show()

+-------+------+------------+------------+--------+-------+
|book_id| title|author_fname|author_lname|username|address|
+-------+------+------------+------------+--------+-------+
|      5|Book E|        Anna|       Davis|       F|   서울|
|      1|Book A|        John|         Doe|       A|   서울|
|      3|Book C|       Emily|       Jones|       C| 경기도|
|      2|Book B|        Jane|       Smith|       B|   대전|
+-------+------+------------+------------+--------+-------+



In [59]:
# books LEFT JOIN users = users RIGHT books

left_join_query = '''
SELECT book_id, title, author_fname, author_lname, username, address
FROM books b LEFT JOIN users u ON b.borrowed_by = u.user_id;
'''
spark.sql(left_join_query).show()

+-------+------+------------+------------+--------+-------+
|book_id| title|author_fname|author_lname|username|address|
+-------+------+------------+------------+--------+-------+
|      4|Book D|       Chris|       Brown|    null|   null|
|      5|Book E|        Anna|       Davis|       F|   서울|
|      1|Book A|        John|         Doe|       A|   서울|
|      3|Book C|       Emily|       Jones|       C| 경기도|
|      2|Book B|        Jane|       Smith|       B|   대전|
+-------+------+------------+------------+--------+-------+



In [61]:
# 사용자의 책 대여 목록 > 전체 사용자 > 대여한 정보가 있으면 나오면, 없으면 NULL 
# books RIGHT JOIN users =  books LEFT JOIN users

right_join_query = '''
SELECT book_id, title, author_fname, author_lname, username, address
FROM books b RIGHT JOIN users u ON b.borrowed_by = u.user_id;
'''
spark.sql(right_join_query).show()

+-------+------+------------+------------+--------+-------+
|book_id| title|author_fname|author_lname|username|address|
+-------+------+------------+------------+--------+-------+
|   null|  null|        null|        null|       G| 경기도|
|      5|Book E|        Anna|       Davis|       F|   서울|
|   null|  null|        null|        null|       I|   부산|
|   null|  null|        null|        null|       E|   null|
|      1|Book A|        John|         Doe|       A|   서울|
|   null|  null|        null|        null|       J|   전주|
|      3|Book C|       Emily|       Jones|       C| 경기도|
|   null|  null|        null|        null|       H|   대구|
|   null|  null|        null|        null|       K|   광주|
|      2|Book B|        Jane|       Smith|       B|   대전|
|   null|  null|        null|        null|       D|   null|
+-------+------+------------+------------+--------+-------+



In [72]:
# 특정지역 = 서울에 거주하는 사용자가 대여한 책 목록

join_query2 = '''
SELECT book_id, title, author_fname, author_lname, username, address
FROM books b INNER JOIN users u ON b.borrowed_by = u.user_id
WHERE u.address = '서울'
ORDER BY b.book_id;
'''
spark.sql(join_query2).show()

+-------+------+------------+------------+--------+-------+
|book_id| title|author_fname|author_lname|username|address|
+-------+------+------------+------------+--------+-------+
|      1|Book A|        John|         Doe|       A|   서울|
|      5|Book E|        Anna|       Davis|       F|   서울|
+-------+------+------------+------------+--------+-------+



In [82]:
# 사용자별로 대여한 책 수 
join_query2 = '''
SELECT user_id, username, count(book_id)
FROM users u INNER JOIN books b ON b.borrowed_by = u.user_id
GROUP BY u.user_id, u.username
ORDER BY u.user_id;
'''
spark.sql(join_query2).show()

+-------+--------+--------------+
|user_id|username|count(book_id)|
+-------+--------+--------------+
|      1|       A|             1|
|      2|       B|             1|
|      3|       C|             1|
|      6|       F|             1|
+-------+--------+--------------+



In [83]:
# book_category,  page > 300 long, short

join_query3 = '''
SELECT book_id, title, CASE WHEN pages>=300 THEN 'Long' ELSE 'Short' END AS book_category
FROM books
'''
spark.sql(join_query3).show()

+-------+------+-------------+
|book_id| title|book_category|
+-------+------+-------------+
|      1|Book A|         Long|
|      2|Book B|        Short|
|      3|Book C|        Short|
|      4|Book D|         Long|
|      5|Book E|        Short|
+-------+------+-------------+



In [87]:
# stock_quantity_category

join_query4 = '''
SELECT book_id, title, stock_quantity,
    CASE 
        WHEN stock_quantity>=50 THEN '충분' 
        WHEN stock_quantity >=30 THEN '보통' 
        ELSE '부족' 
    END AS stock_quantity_category
FROM books;
'''
spark.sql(join_query4).show()

+-------+------+-----------------+-----------------------+
|book_id| title|   stock_quantity|stock_quantity_category|
+-------+------+-----------------+-----------------------+
|      1|Book A|60.50000000000001|                   충분|
|      2|Book B|             44.0|                   보통|
|      3|Book C|             22.0|                   부족|
|      4|Book D|             82.5|                   충분|
|      5|Book E|             38.5|                   보통|
+-------+------+-----------------+-----------------------+



In [89]:
# 책 제목에 특정 키워드가 포함되어 있는지 확인 

join_query5 = '''
SELECT book_id, title
FROM books
WHERE title LIKE '%A%'
'''
spark.sql(join_query5).show()

+-------+------+
|book_id| title|
+-------+------+
|      1|Book A|
+-------+------+



In [108]:
# 가장 많이 대여가 된 책의 작가 조회 

join_query6 = '''
SELECT book_id, author_fname, author_lname, count(book_id) as borrow_count
FROM books
GROUP BY author_fname, author_lname, book_id
ORDER BY borrow_count DESC
'''
spark.sql(join_query6).show()

+-------+------------+------------+------------+
|book_id|author_fname|author_lname|borrow_count|
+-------+------------+------------+------------+
|      1|        John|         Doe|           1|
|      2|        Jane|       Smith|           1|
|      3|       Emily|       Jones|           1|
|      5|        Anna|       Davis|           1|
|      4|       Chris|       Brown|           1|
+-------+------------+------------+------------+



In [142]:
# 책의 발행 연도별 대여 현황: 발행 연도별로 대여된 책의 수를 확인

join_query7 = '''
SELECT released_year, count(book_id) as borrow_count
FROM books
GROUP BY released_year
ORDER BY released_year DESC
'''
spark.sql(join_query7).show()

+-------------+------------+
|released_year|borrow_count|
+-------------+------------+
|         2015|           1|
|         2012|           1|
|         2010|           1|
|         2008|           1|
|         2005|           1|
+-------------+------------+



In [124]:
# 사용자의 지역별 대여된 책 수 : 사용자 지역별로 대여된 책의 수를 계산

join_query8 = '''
SELECT  address, count(b.book_id) as borrow_count
FROM books b RIGHT JOIN users u ON u.user_id = b.borrowed_by
GROUP BY u.address
ORDER BY borrow_count DESC
'''
spark.sql(join_query8).show()

+-------+------------+
|address|borrow_count|
+-------+------------+
|   서울|           2|
|   대전|           1|
| 경기도|           1|
|   전주|           0|
|   부산|           0|
|   null|           0|
|   대구|           0|
|   광주|           0|
+-------+------------+



In [126]:
# 대여되지 않은 책 중에서 페이지 수가 가장 많은 책을 조회합니다.

join_query9 = '''
SELECT book_id, title, pages
FROM books
WHERE borrowed_by IS NULL
'''
spark.sql(join_query9).show()

+-------+------+-----+
|book_id| title|pages|
+-------+------+-----+
|      4|Book D|  320|
+-------+------+-----+



In [131]:
# 재고가 부족한 책과 대여 상태: 재고가 30개 미만인 책과 해당 책이 대여된 상태인지 확인합니다.

join_query10 = '''
SELECT book_id, title, stock_quantity, borrowed_by, 
    CASE
        WHEN borrowed_by IS NOT NULL THEN '대여중'
        ELSE '대여중 아님'
        END AS Stock_status
FROM books
WHERE stock_quantity <30
'''
spark.sql(join_query10).show()

+-------+------+--------------+-----------+------------+
|book_id| title|stock_quantity|borrowed_by|Stock_status|
+-------+------+--------------+-----------+------------+
|      3|Book C|          22.0|          3|      대여중|
+-------+------+--------------+-----------+------------+



In [150]:
# 실행 계획, DAG 형태 분석
spark.sql(join_query8).explain()
spark.sql(join_query9).explain()
spark.sql(join_query10).explain()

== Physical Plan ==
*(7) Sort [borrow_count#1800L DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(borrow_count#1800L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#2344]
   +- *(6) HashAggregate(keys=[address#8], functions=[count(book_id#203L)])
      +- Exchange hashpartitioning(address#8, 200), ENSURE_REQUIREMENTS, [id=#2340]
         +- *(5) HashAggregate(keys=[address#8], functions=[partial_count(book_id#203L)])
            +- *(5) Project [book_id#203L, address#8]
               +- SortMergeJoin [borrowed_by#210L], [user_id#6L], RightOuter
                  :- *(2) Sort [borrowed_by#210L ASC NULLS FIRST], false, 0
                  :  +- Exchange hashpartitioning(borrowed_by#210L, 200), ENSURE_REQUIREMENTS, [id=#2326]
                  :     +- *(1) Project [book_id#203L, borrowed_by#210L]
                  :        +- *(1) Filter isnotnull(borrowed_by#210L)
                  :           +- *(1) Scan ExistingRDD[book_id#203L,title#204,author_fname#205,author_lname#206,pa

In [145]:
# CSV 로 save
books_df.write.csv("data/output/books.csv", header = True, mode = "overwrite")
user_df.write.csv("data/output/users.csv", header = True, mode= "overwrite")

In [ ]:
spark.stop()